# Setting up Project



In [27]:
# log in with the email account associated with your GCP.
import os 
curDir = !pwd
if curDir[0] == '/content':
  from google.colab import auth
  auth.authenticate_user()



In [28]:
!gcloud projects list

PROJECT_ID                    NAME                          PROJECT_NUMBER
billingprj                    BillingPrj                    346241010245
bq-part-clust                 MLA-01-RSDW                   591958418924
dd-sales-funnel               dd-sales-funnel               135191404955
doordash-287907               Doordash                      945652970732
fin-trading                   Financial-Trading             563366445397
ipl-stream                    ipl-stream                    459986636125
masterproject-277322          MasterProject                 799744540190
mlacademy-prj                 mlacademy-prj                 593200915250
qwiklabs-gcp-01-1bda0c438033  qwiklabs-gcp-01-1bda0c438033  859711784674
rsa-app-services              RSA-App-Services              571647258835
rsa-dev-10                    MLA-03-RSA-DEV                777754064427
rsa-mla-prod                  rsa-mla-prod                  84569519344
tpc-ds-01                     tpc-ds-01           

In [29]:
project_id = "qwiklabs-gcp-01-1bda0c438033"
project_id

'qwiklabs-gcp-01-1bda0c438033'

In [30]:
os.environ['PROJECT'] = project_id
!gcloud config set project {project_id}

Updated property [core/project].


In [31]:
os.environ['PROJECT_ID'] = project_id
!echo $PROJECT_ID

qwiklabs-gcp-01-1bda0c438033


# Setup

## Set Default Region

In [32]:
!gcloud config set compute/region us-central1

Updated property [compute/region].


## Create bucket

In [33]:
!gsutil mb -c regional -l us-central1 gs://$PROJECT_ID

Creating gs://qwiklabs-gcp-01-1bda0c438033/...
ServiceException: 409 A Cloud Storage bucket named 'qwiklabs-gcp-01-1bda0c438033' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


# BQ Fundamentals

## Create Dataset

### Simple *creation*

In [34]:
!echo "project_id = {project_id}" > ~/.bigqueryrc


In [35]:
!bq mk --location=US \
      --default_table_expiration 3600 \
      DS01

Dataset 'qwiklabs-gcp-01-1bda0c438033:DS01' successfully created.


### Conditional *creation*

In [36]:
%%bash 
bq_safe_mk() {     
    dataset=$1     
    exists=$(bq ls --datasets | grep -w $dataset)     
    if [ -n "$exists" ]; then        
        echo "Not creating $dataset since it already exists"     
    else        
        echo "Creating $dataset"        
        bq mk $dataset    
    fi
}

bq_safe_mk DS01

bq_safe_mk DS02

Not creating DS01 since it already exists
Creating DS02
Dataset 'qwiklabs-gcp-01-1bda0c438033:DS02' successfully created.


## Create Tables

### Simple Table Creation

In [37]:
!bq mk --table \
    --expiration 3600 \
    --description "Simple Table" \
     --label persistence:volatile \
     DS01.employees employee_id:INTEGER,employee_name:STRING,age:FLOAT

Table 'qwiklabs-gcp-01-1bda0c438033:DS01.employees' successfully created.


### Table creation with schema file

In [38]:
%%writefile sales_schema.json
[
  {
    "description": "quarter",
    "mode": "REQUIRED",
    "name": "qtr",
    "type": "STRING"
  },
  {
    "description": "sales representative",
    "mode": "NULLABLE",
    "name": "rep",
    "type": "STRING"
  },
  {
    "description": "total sales",
    "mode": "NULLABLE",
    "name": "sales",
    "type": "FLOAT"
  }
]


Overwriting sales_schema.json


In [39]:
!bq mk --table \
    --expiration 3600 \
    --description "Sales data" \
     --label persistence:volatile \
     DS01.quarterly_sales_data sales_schema.json

Table 'qwiklabs-gcp-01-1bda0c438033:DS01.quarterly_sales_data' successfully created.


### Table with nested fields

Example schema
The following example shows sample nested and repeated data. This table contains information about people. It consists of the following fields:

- id
- first_name
- last_name
- dob (date of birth)
- addresses (a nested and repeated field)
 - addresses.status (current or previous)
 - addresses.address
 - addresses.city
 - addresses.state
 - addresses.zip
 - addresses.numberOfYears (years at the address

#### Defining Nested & Repeated Schema



In [40]:
%%writefile customer_data_schema.json
[
    {
        "name": "cust_id",
        "type": "STRING",
        "mode": "NULLABLE"
    },
    {
        "name": "first_name",
        "type": "STRING",
        "mode": "NULLABLE"
    },
    {
        "name": "last_name",
        "type": "STRING",
        "mode": "NULLABLE"
    },
    {
        "name": "dob",
        "type": "DATE",
        "mode": "NULLABLE"
    },
    {
        "name": "addresses",
        "type": "RECORD",
        "mode": "REPEATED",
        "fields": [
            {
                "name": "status",
                "type": "STRING",
                "mode": "NULLABLE"
            },
            {
                "name": "address",
                "type": "STRING",
                "mode": "NULLABLE"
            },
            {
                "name": "city",
                "type": "STRING",
                "mode": "NULLABLE"
            },
            {
                "name": "state",
                "type": "STRING",
                "mode": "NULLABLE"
            },
            {
                "name": "zip",
                "type": "STRING",
                "mode": "NULLABLE"
            },
            {
                "name": "numberOfYears",
                "type": "STRING",
                "mode": "NULLABLE"
            }
        ]
    }
]

Overwriting customer_data_schema.json


#### Creating Table with Nested & Repeated Fields



In [41]:
!bq mk --table \
    --expiration 3600 \
    --description "Customer data" \
     --label persistence:volatile \
     DS01.customer_data customer_data_schema.json

Table 'qwiklabs-gcp-01-1bda0c438033:DS01.customer_data' successfully created.


#### Creating Customer data json

In [42]:
%%writefile customer_data_data.json
{"cust_id":"1","first_name":"John","last_name":"Doe","dob":"1968-01-22","addresses":[{"status":"current","address":"123 First Avenue","city":"Seattle","state":"WA","zip":"11111","numberOfYears":"1"},{"status":"previous","address":"456 Main Street","city":"Portland","state":"OR","zip":"22222","numberOfYears":"5"}]}
{"cust_id":"2","first_name":"Jane","last_name":"Doe","dob":"1980-10-16","addresses":[{"status":"current","address":"789 Any Avenue","city":"New York","state":"NY","zip":"33333","numberOfYears":"2"},{"status":"previous","address":"321 Main Street","city":"Hoboken","state":"NJ","zip":"44444","numberOfYears":"3"}]}

Overwriting customer_data_data.json


#### Insert data into nested table

In [43]:
!bq insert DS01.customer_data customer_data_data.json

BigQuery error in insert operation: Table 859711784674:DS01.customer_data is re-
created.


## Load Data to Tables

### Load data to new table (table creation)

#### Load Data from a public dataset

In [44]:
!bq load \
    --source_format=CSV \
    --autodetect \
    DS02.iowa_sales_from_cloud_shell \
    "gs://ven-gc-shared-01/datasets/iowa-liqour-sales/iowa-sales.csv" 

Waiting on bqjob_r7e16f8fbe46d222d_0000017b0f60439c_1 ... (2s) Current status: DONE   


#### Schema of the new table

In [45]:
%%bigquery --project {project_id}
SELECT table_name, column_name,ordinal_position,is_nullable,data_type
 FROM
   DS02.INFORMATION_SCHEMA.COLUMNS
 WHERE
   table_name="iowa_sales_from_cloud_shell"

,table_name,column_name,ordinal_position,is_nullable,data_type
0,iowa_sales_from_cloud_shell,invoice_and_item_number,1,YES,STRING
1,iowa_sales_from_cloud_shell,date,2,YES,DATE
2,iowa_sales_from_cloud_shell,store_number,3,YES,INT64
3,iowa_sales_from_cloud_shell,store_name,4,YES,STRING
4,iowa_sales_from_cloud_shell,address,5,YES,STRING
5,iowa_sales_from_cloud_shell,city,6,YES,STRING
6,iowa_sales_from_cloud_shell,zip_code,7,YES,STRING
7,iowa_sales_from_cloud_shell,store_location,8,YES,STRING
8,iowa_sales_from_cloud_shell,county_number,9,YES,INT64
9,iowa_sales_from_cloud_shell,county,10,YES,STRING


#### Load data to plain tables

In [46]:
!bq load \
    --source_format=CSV \
    --autodetect \
    DS02.iowa_sales_plain \
    "gs://ven-gc-shared-01/datasets/iowa-liqour-sales/iowa-sales.csv" 

Waiting on bqjob_r719eb81baf407bc7_0000017b0f605cbd_1 ... (3s) Current status: DONE   


### Partionted Tables

#### Load data to partionined tables

In [47]:
!bq load \
    --source_format=CSV \
    --time_partitioning_type=DAY \
    --time_partitioning_field date \
    --autodetect \
    DS02.iowa_sales_partitioned \
    "gs://ven-gc-shared-01/datasets/iowa-liqour-sales/iowa-sales.csv" 

Waiting on bqjob_r36d04c2a2e35a369_0000017b0f6072b8_1 ... (34s) Current status: DONE   


#### Querrying Partioned tables

In [48]:
%%bigquery --project {project_id}
SELECT * FROM `DS02.iowa_sales_plain`where date BETWEEN '2012-01-01' AND '2012-01-31';

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,S03651300006,2012-01-19,2190,"Central City Liquor, Inc.",1460 2ND AVE,DES MOINES,50314,POINT (-93.619787 41.60566),77,Polk,1062050,BARBADOS RUM,91,Castle Brands,902144,Gosling's Black Seal Rum,12,750,8.50,12.75,12,153.00,9.00,2.38
1,S03385800026,2012-01-03,3554,Dick's Beverage Depot,102 N MAIN,TRIPOLI,50676,POINT (-92.258152 42.809882),9,Bremer,1081355,STRAWBERRY SCHNAPPS,434,Luxco-St Louis,84496,Paramount Strawberry Schnapps,12,750,3.97,5.95,12,71.40,9.00,2.38
2,S03694500034,2012-01-23,2582,Hy-Vee #2 Food Store / Mason City,551 S ILLINOIS AVE,MASON CITY,50401,POINT (-93.17114 43.14623),17,Cerro Gordo,1081355,STRAWBERRY SCHNAPPS,434,Luxco-St Louis,84496,Paramount Strawberry Schnapps,12,750,3.97,5.95,12,71.40,9.00,2.38
3,S03471300017,2012-01-09,4189,Fareway Stores #594 / Manchester,109 EAST MARION ST,MANCHESTER,52057,POINT (-91.457225 42.48217),28,Delaware,1081230,WHITE CREME DE CACAO,434,Luxco-St Louis,78866,Paramount Creme De Cacao/white,12,750,4.44,6.66,12,79.92,9.00,2.38
4,S03688300027,2012-01-23,4150,Fareway Stores #951 / Waterloo,40 W SAN MARNAN,WATERLOO,50701,None,7,Black Hawk,1081230,WHITE CREME DE CACAO,434,Luxco-St Louis,78616,Arrow Creme de Cacao White,12,750,4.39,6.58,12,78.96,9.00,2.38
5,S03414400004,2012-01-04,4485,DYNO'S 51 / SANBORN,101 W 7TH ST,SANBORN,51248,POINT (-95.656064 43.186035000000004),71,O'Brien,1051110,BLACKBERRY BRANDIES,434,Luxco-St Louis,55086,Paramount Blackberry Brandy,12,750,5.21,7.82,12,93.84,9.00,2.38
6,S03479000094,2012-01-09,2621,Hy-Vee Food Store #3 / Sioux City,3301 GORDON DR,SIOUX CITY,51105,POINT (-96.364108 42.487752),97,Woodbury,1051110,BLACKBERRY BRANDIES,380,Phillips Beverage Company,55106,Phillips Blackberry Flav Brandy,12,750,4.75,7.13,12,85.56,9.00,2.38
7,S03456700095,2012-01-05,2190,"Central City Liquor, Inc.",1460 2ND AVE,DES MOINES,50314,POINT (-93.619787 41.60566),77,Polk,1081700,DISTILLED SPIRITS SPECIALTY,434,Luxco-St Louis,75087,Juarez Gold Dss,12,1000,4.75,7.13,60,427.80,60.00,15.85
8,S03773800029,2012-01-26,3890,Smokin' Joe's #7 Tobacco and Liquor,904 1ST AVE NW,CEDAR RAPIDS,52405,POINT (-91.683669 41.972773),57,Linn,1051100,APRICOT BRANDIES,259,Heaven Hill Brands,52314,Christian Bros Brandy,24,375,2.93,4.40,5,22.00,1.88,0.50
9,S03446100027,2012-01-05,3797,The Liquor Store / Davenport,211 W 53RD ST,DAVENPORT,52806,POINT (-90.57611300000002 41.574536),82,Scott,1081312,BUTTERSCOTCH SCHNAPPS,65,Jim Beam Brands,82784,Dekuyper Buttershots Burst Schnapps,24,375,3.06,4.59,24,110.16,9.00,2.38


In [49]:
%%bigquery --project {project_id}
SELECT * FROM `DS02.iowa_sales_partitioned` where date BETWEEN '2012-01-01' AND '2012-01-31';

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,S03522700054,2012-01-11,2644,Hy-Vee Fort Dodge Wine and Spirits,1511 2ND AVE NORTH,FORT DODGE,50501,POINT (-94.177165 42.508344),94,Webster,1081380,MISCELLANEOUS SCHNAPPS,65,Jim Beam Brands,82626,Dekuyper Cherry Pucker Schnapps,12,750,6.30,9.45,12,113.40,9.00,2.38
1,S03544400059,2012-01-12,2106,Hillstreet News and Tobacco,2217 COLLEGE,CEDAR FALLS,50613,POINT (-92.455796 42.517182),7,Black Hawk,1081317,GRAPE SCHNAPPS,65,Jim Beam Brands,82636,Dekuyper Grape Pucker Schnapps,12,750,6.30,9.45,132,1247.40,99.00,26.15
2,S03816900136,2012-01-30,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14TH ST,DES MOINES,50320,POINT (-93.596754 41.554101),77,Polk,1081317,GRAPE SCHNAPPS,65,Jim Beam Brands,82637,Dekuyper Grape Pucker,12,1000,7.35,11.02,96,1057.92,96.00,25.36
3,S03813700022,2012-01-30,2564,Hy-Vee Food Store #4 / Waterloo,4000 UNIVERSITY,WATERLOO,50701,POINT (-92.403843 42.505197),7,Black Hawk,1081700,DISTILLED SPIRITS SPECIALTY,421,"Sazerac Co., Inc.",77487,Tortilla Gold Dss,12,1000,4.51,6.76,120,811.20,120.00,31.70
4,S03702100010,2012-01-23,2190,"Central City Liquor, Inc.",1460 2ND AVE,DES MOINES,50314,POINT (-93.619787 41.60566),77,Polk,1081400,TRIPLE SEC,434,Luxco-St Louis,86251,Juarez Triple Sec,12,1000,2.21,3.31,48,158.88,48.00,12.68
5,S03674200017,2012-01-23,3524,Sam's Club 6568 / Ames,305 AIRPORT RD,AMES,50010,POINT (-93.613648 42.001123),85,Story,1081400,TRIPLE SEC,434,Luxco-St Louis,86251,Juarez Triple Sec,12,1000,2.21,3.31,84,278.04,84.00,22.19
6,S03688200064,2012-01-23,3692,Wilkie Liquors,724 1ST ST E,MT VERNON,52314,POINT (-91.960918 41.994458),57,Linn,1051110,BLACKBERRY BRANDIES,434,Luxco-St Louis,54646,Arrow Blackberry Flav Brandy,12,750,5.00,7.50,12,90.00,9.00,2.38
7,S03694500034,2012-01-23,2582,Hy-Vee #2 Food Store / Mason City,551 S ILLINOIS AVE,MASON CITY,50401,POINT (-93.17114 43.14623),17,Cerro Gordo,1081355,STRAWBERRY SCHNAPPS,434,Luxco-St Louis,84496,Paramount Strawberry Schnapps,12,750,3.97,5.95,12,71.40,9.00,2.38
8,S03688300027,2012-01-23,4150,Fareway Stores #951 / Waterloo,40 W SAN MARNAN,WATERLOO,50701,None,7,Black Hawk,1081230,WHITE CREME DE CACAO,434,Luxco-St Louis,78616,Arrow Creme de Cacao White,12,750,4.39,6.58,12,78.96,9.00,2.38
9,S03824600004,2012-01-31,2624,Hy-Vee #2 / Dubuque,2395 NW ARTERIAL RD,DUBUQUE,52002,POINT (-90.741064 42.516633),31,Dubuque,1101100,AMERICAN ALCOHOL,434,Luxco-St Louis,41846,Everclear Alcohol,12,750,8.02,12.03,12,144.36,9.00,2.38


### Clustering Tables


#### Load data into clustering columns

In [50]:
!bq load \
    --source_format=CSV \
    --time_partitioning_type=DAY \
    --time_partitioning_field date \
    --clustering_fields vendor_name,item_description \
    --autodetect \
    DS02.iowa_sales_clustered \
    "gs://ven-gc-shared-01/datasets/iowa-liqour-sales/iowa-sales.csv " 

Waiting on bqjob_r3eb119cfeca155cd_0000017b0f610d16_1 ... (34s) Current status: DONE   


#### Querying Clustered Tables

In [51]:
%%bigquery --project {project_id}
SELECT vendor_name,item_description,sum(sale_dollars) as sales 
FROM `DS02.iowa_sales_partitioned` 
where date BETWEEN '2012-01-01' AND '2012-01-31'
group by vendor_name,item_description;

,vendor_name,item_description,sales
0,Jim Beam Brands,Dekuyper Luscious Razzmatazz,113.40
1,Phillips Beverage Company,Phillips Blackberry Flav Brandy,85.56
2,Luxco-St Louis,Paramount Creme De Cacao/white,79.92
3,Luxco-St Louis,Paramount Amaretto,285.30
4,Jim Beam Brands,Dekuyper Grape Pucker,2248.08
5,"Sazerac Co., Inc.",Tortilla Gold Dss,4380.48
6,Luxco-St Louis,Everclear Alcohol,144.36
7,Jim Beam Brands,Dekuyper Luscious Peachtree Schnapps,226.80
8,Pernod Ricard USA/Austin Nichols,Kahlua Coffee Liqueur,1804.78
9,Castle Brands,Gosling's Black Seal Rum,153.00


In [52]:
%%bigquery --project {project_id}
SELECT vendor_name,item_description,sum(sale_dollars) as sales 
FROM `DS02.iowa_sales_clustered` 
where date BETWEEN '2012-01-01' AND '2012-01-31'
group by vendor_name,item_description;

,vendor_name,item_description,sales
0,Jim Beam Brands,Dekuyper Luscious Peachtree Schnapps,226.80
1,Pernod Ricard USA/Austin Nichols,Kahlua Coffee Liqueur,1804.78
2,Jim Beam Brands,Dekuyper Cherry Pucker Schnapps,113.40
3,Luxco-St Louis,Juarez Triple Sec,635.52
4,Luxco-St Louis,Arrow Blackberry Flav Brandy,180.00
5,Luxco-St Louis,Paramount Strawberry Schnapps,142.80
6,Luxco-St Louis,Arrow Creme de Cacao White,78.96
7,Jim Beam Brands,Dekuyper Sour Apple Pucker,113.40
8,Jim Beam Brands,Dekuyper Peachtree Schnapps Traveler,113.40
9,Jim Beam Brands,Dekuyper Peachtree Schnapps,22.95
